In [1]:
!pip install sympy
!pip install antlr4-python3-runtime==4.10
!pip install latex2sympy2

  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/d2/05/e6600db80270777c4a64238a98d442f0fd07cc8915be2a1c16da7f2b9e74/sympy-1.12-py3-none-any.whl.metadata
  Obtaining dependency information for mpmath>=0.19 from https://files.pythonhosted.org/packages/43/e3/7d92a15f894aa0c9c4b49b8ee9ac9850d6e63b03c9c32c0367a13ae62209/mpmath-1.3.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 497.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 437.9 kB/s eta 0:00:00a 0:00:01
  Using cached antlr4_python3_runtime-4.10-py3-none-any.whl
  Attempting uninstall: antlr4-python3-runtime
    Found existing installation: antlr4-python3-runtime 4.7.2
    Uninstalling antlr4-python3-runtime-4.7.2:
      Successfully uninstalled antlr4-python3-runtime-4.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follow

In [22]:
import sympy
from sympy.parsing.latex import parse_latex
from sympy import Symbol, Integer, Pow, Mul, Add, srepr, symbols
import networkx as nx
import qnet
from sympy import sympify, S
from sympy.parsing.latex import parse_latex
from sympy.parsing.sympy_parser import parse_expr

In [6]:
x, y, z = symbols('x y z')

In [13]:
expr = x**2 + x*y + 5*z**3 
pseudo_tree = srepr(expr)
print(type(pseudo_tree))

<class 'str'>


- adciona o argumento
- se abrir parênteses, adciona um nó na esquerda
- conta a profundidade q ta na árvore, para voltar depois
- da para contar a profundidade com o número de parênteses que abriu e fechou, sem considerar o parênteses de Integer e Symbol

In [10]:
import ast

expr = x**2 + x*y + 5*z**3
pseudo_tree = srepr(expr)

class Node:
    def __init__(self, name):
        self.name = name
        self.children = []

def srepr_to_tree(srepr):
    tree = ast.parse(srepr).body[0].value
    return build_tree(tree)

def build_tree(node):
    if isinstance(node, ast.Name):
        return Node(node.id)
    elif isinstance(node, ast.Call):
        parent = Node(node.func.id)
        for arg in node.args:
            parent.children.append(build_tree(arg))
        return parent
    elif isinstance(node, ast.Constant):
        return Node(repr(node.value))  # Use repr() to get the string representation of the value
    else:
        return Node(str(node))

def print_tree(node, indent=0):
    print("  " * indent + node.name)
    for child in node.children:
        print_tree(child, indent + 1)

tree = srepr_to_tree(pseudo_tree)
print_tree(tree)

Add
  Pow
    Symbol
      'x'
    Integer
      2
  Mul
    Symbol
      'x'
    Symbol
      'y'
  Mul
    Integer
      5
    Pow
      Symbol
        'z'
      Integer
        3


In [25]:
from sympy import symbols, S
from sympy.core.basic import Basic

# Define the symbols
x, y, z = symbols('x y z')

# Define the expression
expr = x**2 + x*y + 5*z**3

# Define a simple Tree node
class TreeNode:
    def __init__(self, value):
        self.value = value  # The function or value the node represents
        self.children = []  # List of child nodes

    def add_child(self, child):
        self.children.append(child)

    def __repr__(self, level=0):
        ret = "\t"*level+repr(self.value)+"\n"
        for child in self.children:
            ret += child.__repr__(level+1)
        return ret

# Function to build a tree from a SymPy expression
def build_tree(expr, parent=None):
    if isinstance(expr, Basic):
        if expr.is_Atom:
            # If the expression is an atom (i.e., cannot be decomposed further), it becomes a leaf node
            node = TreeNode(str(expr))
        else:
            # If the expression is not an atom, it is a function or operation with arguments
            node = TreeNode(str(expr.func))
            for arg in expr.args:
                child_node = build_tree(arg, node)
                node.add_child(child_node)
    else:
        # For handling numbers and other simple types not covered by Basic
        node = TreeNode(str(expr))
    return node

# Build the tree from the expression
tree = build_tree(expr)
print(tree)


"<class 'sympy.core.add.Add'>"
	"<class 'sympy.core.power.Pow'>"
		'x'
		'2'
	"<class 'sympy.core.mul.Mul'>"
		'5'
		"<class 'sympy.core.power.Pow'>"
			'z'
			'3'
	"<class 'sympy.core.mul.Mul'>"
		'x'
		'y'



In [26]:
from sympy import simplify, srepr, Eq
from latex2sympy2 import latex2sympy, latex2latex
from difflib import SequenceMatcher


def simplify_latex_expression(latex_expr):
    return latex2sympy(latex2latex(latex_expr))

def simplify_sympy_expression(sympy_expr):
    return simplify(sympy_expr.doit().doit())

def simpy_to_tree(sympy_expr):
    return srepr(simplify_sympy_expression(sympy_expr))

def latex_to_tree(latex_expr):
    return srepr(simplify_latex_expression(latex_expr))


In [44]:
simplify_latex_expr = simplify_latex_expression('$x+y^2+x$')

In [48]:
from sympy import sympify
from sympy.core.basic import Basic

# Input string representing the expression in SymPy format
input_str = "Add(Mul(Integer(2), Symbol('x')), Pow(Symbol('y'), Integer(2)))"

# Evaluate the string to get the SymPy expression object
expr = sympify(input_str)

# Define a simple Tree node
class TreeNode:
    def __init__(self, value):
        # If the value is a class, extract just the class name (function name)
        if isinstance(value, Basic):
            self.value = value.__class__.__name__
        else:
            self.value = value
        self.children = []  # List of child nodes

    def add_child(self, child):
        self.children.append(child)

    def __repr__(self, level=0):
        ret = "\t"*level+repr(self.value)+"\n"
        for child in self.children:
            ret += child.__repr__(level+1)
        return ret

# Function to build a tree from a SymPy expression
def build_tree(expr, parent=None):
    if isinstance(expr, Basic):
        if expr.is_Atom:
            # If the expression is an atom (i.e., cannot be decomposed further), it becomes a leaf node
            node = TreeNode(str(expr))
        else:
            # If the expression is not an atom, it is a function or operation with arguments
            node = TreeNode(expr.func)  # Pass the func directly
            for arg in expr.args:
                child_node = build_tree(arg, node)
                node.add_child(child_node)
    else:
        # For handling numbers and other simple types not covered by Basic
        node = TreeNode(str(expr))
    return node

# Build the tree from the expression
tree = build_tree(expr)
print(tree)

<class 'sympy.core.add.Add'>
	<class 'sympy.core.power.Pow'>
		'y'
		'2'
	<class 'sympy.core.mul.Mul'>
		'2'
		'x'

